In [2]:
import os

os.environ['PATH'] = f'{os.environ["PATH"]}:/home/fran/IronhackMad/dataptmad1019/module-1/lab-selenium/driver'

In [ ]:
os.environ['PATH']

In [244]:
from selenium import webdriver
import re
import pandas as pd
import numpy as np

In [4]:
import datetime

ORIGIN = 'MADRID'
DESTINATION = 'VALENCIA'

# tickets for tomorrow
DATE = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%d/%m/%Y')

In [47]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests

browser = webdriver.Firefox()
browser.get('http://www.renfe.com/')

origin = browser.find_element_by_name('desOrigen')
origin.send_keys(ORIGIN)
origin.send_keys(Keys.ARROW_DOWN)
origin.send_keys(Keys.ENTER)

destination = browser.find_element_by_name('desDestino')
destination.send_keys(DESTINATION)
destination.send_keys(Keys.ARROW_DOWN)
destination.send_keys(Keys.ENTER)

date = browser.find_element_by_name('__fechaIdaVisual')
date.send_keys(Keys.BACK_SPACE * 10)
date.send_keys(DATE)

destination.send_keys(Keys.ENTER)

element = WebDriverWait(browser, 60).until(EC.presence_of_element_located((By.ID, "listaTrenesTBodyIda")))
new_url = browser.current_url
print(new_url)
#html = requests.get(new_url).content
#soup = BeautifulSoup(html, "lxml")
#table_id = browser.find_element(By.ID, 'listaTrenesTBodyIda')
target_table = browser.find_element_by_xpath("//*[@id='listaTrenesTBodyIda']")
time.sleep(4)
#rows = target_table.find_elements(By.TAG_NAME, 'tr')
rows_trains = target_table.find_elements_by_xpath("//*[contains(@id,'tren')]")
rows_parsed = [i.get_attribute('innerHTML') for i in rows_trains]
#for i in rows_trains:
    #rows_parsed.append(i.get_attribute('innerHTML'))
    
browser.close()

https://venta.renfe.com/vol/search.do?c=_OveS


In [111]:
def departure(x):
    return re.findall('salida\">(\d+.\d+)', x)
def arrival(x):
    return re.findall('llegada\">(\d+.\d+)', x)
def duration(x):
    return re.findall('span\>(\d \w. \d* min)', x)
def service(x):
    return re.findall('span\>([A-Z]{2,}-?[A-Z]*)', x)
def price(x):
    return re.findall('alse\">(\d+,?\d*)', x)
def ticket_class(x):
    return re.findall('clase\d*\">(\w*)', x)
def ticket_offer(x):
    return re.findall('ifa\d+\">(\w+ ?\w*)', x)

In [212]:
departures = [departure(i) for i in rows_parsed]

In [246]:
arrivals = [arrival(i) for i in rows_parsed]
durations = [duration(i) for i in rows_parsed]
services = [service(i) for i in rows_parsed]
prices = [price(i) for i in rows_parsed]
ticket_classes = [ticket_class(i) for i in rows_parsed]
ticket_offers = [ticket_offer(i) for i in rows_parsed]

In [247]:
df_renfe = pd.DataFrame(
    {'departure': departures,
     'arrival': arrivals,
     'duration': durations,
     'service': services,
     'price': prices,
     'ticket_class': ticket_classes,
     'ticket_offer': ticket_offers
    })

In [204]:
col_names = df_renfe.columns.values.tolist()

In [249]:
def replace_brackets(x, y):
    return x[y].astype('str').str.replace('\[|\]|\'', '')

In [250]:
for i in col_names:
    df_renfe[i] = replace_brackets(df_renfe,i)

In [262]:
null_df_renfe = df_renfe.replace('', np.NaN)

In [264]:
cleaned_df = null_df_renfe.dropna(how='all').reset_index(drop=True)

In [266]:
cleaned_df.to_csv('../processed/renfe_table.csv', index=False)